In [23]:
!pip install tflearn

    100% |████████████████████████████████| 102kB 705kB/s a 0:00:01
  Running setup.py bdist_wheel for tflearn ... done
  Stored in directory: /home/semih/.cache/pip/wheels/fb/06/72/0478c938ca315c6fddcce8233b80ec91a115ce4496a27e8c90
Successfully built tflearn


In [1]:
%matplotlib inline
import tflearn

import pickle
import tensorflow as tf
import pandas as pd
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
from six.moves import cPickle as pickle
import os
import platform
from subprocess import check_output
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation


/home/semih/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def var(name, shape, init=None, std=None):
    if init is None:
        if std is None:
            std = (2./shape[0])**0.5
        init = tf.truncated_normal_initializer(stddev=std)
    return tf.get_variable(name=name, shape=shape, 
                           dtype=tf.float32, initializer=init)

In [3]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo,  encoding='latin1')
    return dict

In [4]:
# Load data
tra_file_1 = './cifar-10-batches-py/data_batch_1'
tra_file_2 = './cifar-10-batches-py/data_batch_2'
tra_file_3 = './cifar-10-batches-py/data_batch_3'
tra_file_4 = './cifar-10-batches-py/data_batch_4'
tra_file_5 = './cifar-10-batches-py/data_batch_5'

test_file = './cifar-10-batches-py/test_batch'

x = None 
y = None 

for i in range(5):
    if i == 0: 
        tra_batch = unpickle(tra_file_1)
    elif i == 1:
        tra_batch = unpickle(tra_file_2)
    elif i == 2:
        tra_batch = unpickle(tra_file_3)
    elif i == 3:
        tra_batch = unpickle(tra_file_4)
    elif i == 4:
        tra_batch = unpickle(tra_file_5)
        
    _X = tra_batch["data"]
    _Y = tra_batch['labels']

    _X = np.array(_X, dtype=float) / 255.0
    _X = _X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
    #_X = _X.reshape([-1, 3, 32, 32])
    _X = _X.transpose([0, 2, 3, 1])
    _X = _X.reshape(-1, 32*32*3)
    _Y = np.array(_Y)

    if x is None:
        x = _X
        y = _Y
    else:
        x = np.concatenate((x, _X), axis=0)
        y = np.concatenate((y, _Y), axis=0)
    
    
test_x = None 
test_y = None 

test_batch = unpickle(test_file)
        
_X = test_batch["data"]
_Y = test_batch['labels']

_X = np.array(_X, dtype=float) / 255.0
#_X = _X.reshape([-1, 3, 32, 32])
_X = _X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
_X = _X.transpose([0, 2, 3, 1])
_X = _X.reshape(-1, 32*32*3)


x_test = _X
y_test = _Y

In [5]:
"""def load_pickle(f):
    version = platform.python_version_tuple()
    if version[0] == '2':
        return  pickle.load(f)
    elif version[0] == '3':
        return  pickle.load(f, encoding='latin1')
    raise ValueError("invalid python version: {}".format(version))

def load_CIFAR_batch(filename):
    """ load single batch of cifar """
    with open(filename, 'rb') as f:
        datadict = load_pickle(f)
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
        Y = np.array(Y)
        return X, Y

def load_CIFAR10(ROOT):
    """ load all of cifar """
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte
"""

SyntaxError: invalid syntax (<ipython-input-5-87125e3f5c70>, line 10)

In [8]:
len(x)
x = x.reshape(50000, 3, 32, 32).transpose(0,2,3,1).astype("float")
x.shape

(50000, 32, 32, 3)

In [5]:
_IMAGE_SIZE = 32
_IMAGE_CHANNELS = 3
_NUM_CLASSES = 10

In [18]:
g = tf.Graph()
with g.as_default():
    X = tf.placeholder(dtype=tf.float32,shape=[None, _IMAGE_SIZE * _IMAGE_SIZE * _IMAGE_CHANNELS])
    Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])
    
    # hidden layer
    W1 = var('W1',[_IMAGE_SIZE * _IMAGE_SIZE * _IMAGE_CHANNELS, 1000])
    b1 = var('b1',[1000],tf.constant_initializer(0.1))
    out1 = tf.nn.relu(tf.matmul(X, W1) + b1)

    #output layer
    W2 = var('W2',[1000, 10])
    b2 = var('b2',[10],tf.constant_initializer(0.1))
    logits = tf.matmul(out1, W2) + b2
    
    #accuracy
    pred = tf.argmax(logits, axis=1)
    truth = tf.argmax(Y, axis=1)
    match = tf.cast(tf.equal(pred, truth), tf.float32)
    acc = tf.reduce_mean(match)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=Y))
    
    step = tf.train.AdamOptimizer().minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [27]:
g2 = tf.Graph()
with g2.as_default():
    X = tf.placeholder(dtype=tf.float32,shape=[None, _IMAGE_SIZE * _IMAGE_SIZE * _IMAGE_CHANNELS])
    Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])
    
    # hidden layer
    W1 = var('W1',[_IMAGE_SIZE * _IMAGE_SIZE * _IMAGE_CHANNELS, 1000])
    b1 = var('b1',[1000],tf.constant_initializer(0.1))
    out1 = tf.nn.relu(tf.matmul(X, W1) + b1)

    #output layer
    W2 = var('W2',[1000, 10])
    b2 = var('b2',[10],tf.constant_initializer(0.1))
    logits = tf.matmul(out1, W2) + b2
    
    #accuracy
    pred = tf.argmax(logits, axis=1)
    truth = tf.argmax(Y, axis=1)
    match = tf.cast(tf.equal(pred, truth), tf.float32)
    acc = tf.reduce_mean(match)
    
    # Declare Lasso loss function
    # Lasso Loss = L2_Loss + heavyside_step,
    # Where heavyside_step ~ 0 if A < constant, otherwise ~ 99
    lasso_param = tf.constant(0.9)
    heavyside_step = tf.truediv(1., tf.add(1., tf.exp(tf.multiply(-50., tf.subtract(X, lasso_param)))))
    regularization_param = tf.multiply(heavyside_step, 99.)
    loss = tf.add(tf.reduce_mean(tf.square(Y - logits)), regularization_param)
    
    step = tf.train.AdamOptimizer().minimize(loss)

In [6]:
g4 = tf.Graph()
with g4.as_default():
    Wconv1 = tf.get_variable("Wconv1", shape=[5, 5, 3, 32])
    bconv1 = tf.get_variable("bconv1", shape=[32])
    Wconv2 = tf.get_variable("Wconv2", shape=[5, 5, 32, 64])
    bconv2 = tf.get_variable("bconv2", shape=[64])
    # affine layer with 1024
    W1 = tf.get_variable("W1", shape=[3136, 1024])
    b1 = tf.get_variable("b1", shape=[1024])
    #affine layer with 10
    W2 = tf.get_variable("W2", shape=[1024, 10])
    b2 = tf.get_variable("b2", shape=[10])    
    X = tf.placeholder(dtype=tf.float32,shape=[None,32, 32, 3])
    Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])
    conv1 = tf.nn.conv2d(X, Wconv1, strides=[1, 1, 1, 1], padding='SAME') + bconv1
    
    # ReLU Activation Layer
    relu1 = tf.nn.relu(conv1)
    
    # Conv
    conv2 = tf.nn.conv2d(relu1, Wconv2, strides=[1, 2, 2, 1], padding='VALID') + bconv2
    
    # ReLU Activation Layer
    relu2 = tf.nn.relu(conv2)
    
    # 2x2 Max Pooling layer with a stride of 2
    maxpool = tf.layers.max_pooling2d(relu2, pool_size=(2,2), strides=2)
    
    maxpool_flat = tf.reshape(maxpool,[-1,3136])
    # Spatial Batch Normalization Layer (trainable parameters, with scale and centering)
    bn1 = tf.layers.batch_normalization(inputs=maxpool_flat, center=True, scale=True, training=True)
    # Affine layer with 1024 output units
    affine1 = tf.matmul(bn1, W1) + b1
    
    # vanilla batch normalization
    affine1_flat = tf.reshape(affine1,[-1,1024])
    bn2 = tf.layers.batch_normalization(inputs=affine1, center=True, scale=True, training=True)
    
    # ReLU Activation Layer
    relu2 = tf.nn.relu(bn2)

    # dropout
    drop1 = tf.layers.dropout(inputs=relu2, training=True)
    # Affine layer from 1024 input units to 10 outputs
    affine2 = tf.matmul(drop1, W2) + b2
    # vanilla batch normalization
    affine2_flat = tf.reshape(affine2,[-1,3136])
    pred = tf.layers.batch_normalization(inputs=affine2, center=True, scale=True, training=True)
    prediction = tf.nn.softmax(pred)
    pred2 = tf.argmax(prediction, axis=1)
    truth = tf.argmax(Y, axis=1)
    match = tf.cast(tf.equal(pred2, truth), tf.float32)
    
    acc = tf.reduce_mean(match)

    lasso_param = tf.constant(0.9)
    heavyside_step = tf.truediv(1., tf.add(1., tf.exp(tf.multiply(-50., tf.subtract(X, lasso_param)))))
    regularization_param = tf.multiply(heavyside_step, 99.)
    loss = tf.add(tf.reduce_mean(tf.square(Y - pred)), regularization_param)
    
    step = tf.train.AdamOptimizer().minimize(loss)

In [31]:
g3 = tf.Graph()
with g3.as_default():
    X = tf.placeholder(dtype=tf.float32,shape=[None, _IMAGE_SIZE * _IMAGE_SIZE * _IMAGE_CHANNELS])
    Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])
    
    
    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    #accuracy
    pred = tf.argmax(logits, axis=1)
    truth = tf.argmax(Y, axis=1)
    match = tf.cast(tf.equal(pred, truth), tf.float32)
    acc = tf.reduce_mean(match)
    
    # Declare Lasso loss function
    # Lasso Loss = L2_Loss + heavyside_step,
    # Where heavyside_step ~ 0 if A < constant, otherwise ~ 99
    lasso_param = tf.constant(0.9)
    heavyside_step = tf.truediv(1., tf.add(1., tf.exp(tf.multiply(-50., tf.subtract(X, lasso_param)))))
    regularization_param = tf.multiply(heavyside_step, 99.)
    loss = tf.add(tf.reduce_mean(tf.square(Y - logits)), regularization_param)
    
    step = tf.train.AdamOptimizer().minimize(loss)

NameError: name 'mode' is not defined

In [ ]:
# Select appropriate loss function based on regression type

if regression_type == 'LASSO':
    # Declare Lasso loss function
    # Lasso Loss = L2_Loss + heavyside_step,
    # Where heavyside_step ~ 0 if A < constant, otherwise ~ 99
    lasso_param = tf.constant(0.9)
    heavyside_step = tf.truediv(1., tf.add(1., tf.exp(tf.multiply(-50., tf.subtract(A, lasso_param)))))
    regularization_param = tf.multiply(heavyside_step, 99.)
    loss = tf.add(tf.reduce_mean(tf.square(y_target - model_output)), regularization_param)

elif regression_type == 'Ridge':
    # Declare the Ridge loss function
    # Ridge loss = L2_loss + L2 norm of slope
    ridge_param = tf.constant(1.)
    ridge_loss = tf.reduce_mean(tf.square(A))
    loss = tf.expand_dims(tf.add(tf.reduce_mean(tf.square(y_target - model_output)), tf.multiply(ridge_param, ridge_loss)), 0)
    
else:
    print('Invalid regression_type parameter value',file=sys.stderr)


In [7]:
def getBatch(x, y, batch_size = 10000, n=0):
    if n == 0:
        x = x[:batch_size]
    else:
        x = x[batch_size * n: batch_size * (n + 1) ]
    x = x.reshape(batch_size, 3, 32, 32).transpose(0,2,3,1).astype("float")
    lst_labels = list()
    lst_temp = list()
    for i in range(batch_size):
        lst_temp = list()
        for j in range(10):
            if y[i] == j:
                lst_temp.append(1)
            else:
                lst_temp.append(0)
        lst_labels.append(lst_temp)
    return x, np.asarray(lst_labels)


In [9]:
x_tra, y_tra = getBatch(x, y, batch_size=1000)

In [10]:
labels = np.asarray(lst_labels)

NameError: name 'lst_labels' is not defined

In [11]:
labels = y_tra

In [12]:
sess = tf.InteractiveSession(graph=g4)
tf.global_variables_initializer().run()

In [13]:
for I in range(1000):
    # Train for batches 
    for j in range(50):
        x_tra, y_tra = getBatch(x,y, batch_size=1000, n=j)
        _, acc_ = sess.run([step, acc], feed_dict={X:x_tra, Y:labels})
        print('%d) acc: %2.4f' % (I,acc_), end='\r')
    
    

KeyboardInterrupt: 

In [28]:
_, t_acc_ = sess.run( [step, acc], feed_dict={X:x_test2, Y:y_test2})

In [30]:
_, acc_ = sess.run([step, acc], feed_dict={X:x_tra, Y:labels})


In [32]:
print('%d) acc: %2.4f' % (I,t_acc_), end='\r')
#print('%d) acc: %2.4f' % (I,acc_), end='\r')

In [9]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, 32, 32, 3])
y = tf.placeholder(tf.int64, [None])
is_training = tf.placeholder(tf.bool)

net = CifarNet()
net.forward(X,y,is_training)

(?, 32, 32, 32)
Tensor("Relu:0", shape=(?, 32, 32, 32), dtype=float32)
(?, 14, 14, 64)
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
(?, 7, 7, 64)
(?, 1024)
(?, 1024)
(?, 1024)
(?, 10)


<tf.Tensor 'batch_normalization_3/cond/Merge:0' shape=(?, 10) dtype=float32>

In [10]:
# Annealing the learning rate
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 1e-3
end_learning_rate = 5e-3
decay_steps = 10000

learning_rate = tf.train.polynomial_decay(starter_learning_rate, global_step,
                                          decay_steps, end_learning_rate,
                                          power=0.5)

exp_learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                               100000, 0.96, staircase=True)


# Feel free to play with this cell
mean_loss = None
optimizer = None

# define our loss
cross_entr_loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(y,10), logits=net.predict)
mean_loss = tf.reduce_mean(cross_entr_loss)

# define our optimizer
optimizer = tf.train.AdamOptimizer(exp_learning_rate)


# batch normalization in tensorflow requires this extra dependency
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_update_ops):
    train_step = optimizer.minimize(mean_loss, global_step=global_step)

In [11]:
# train with 10 epochs
sess = tf.Session()

try:
    with tf.device("/gpu:0") as dev:
        sess.run(tf.global_variables_initializer())
        print('Training')
        net.run(sess, mean_loss, X_train, y_train, 10, 64, 200, train_step, True)
        print('Validation')
        net.run(sess, mean_loss, X_val, y_val, 1, 64)
except tf.errors.InvalidArgumentError:
    print("no gpu found, please use Google Cloud if you want GPU acceleration") 
net.run(sess, mean_loss, X_train, y_train, 10, 64, 200, train_step, True)
    

no gpu found, please use Google Cloud if you want GPU acceleration


InvalidArgumentError: Node 'ArgMax': Unknown input node 'batch_normalization_3/cond/Merge'

In [43]:
x_test2, y_test2= getBatch(x,y, batch_size=10000, n=4)

In [29]:
x_test2 = x
y_test2 = y

In [24]:
y_test2.shape

(10000, 10)

In [44]:
for i in range(200):
    print(y_test2[i])

[0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 1 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 1 0]
[0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 1 0 0]
[0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 1]
[0 0 0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 1 0 0 0]
[0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0]
[0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0]
[0 0 0 1 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0]
[0 1 0 0 0 0 0 0 0 0]
[0 1 0 0 0

In [ ]:
test_batch = unpickle(test_file)
        
_X = test_batch["data"]
_Y = test_batch['labels']

_X = np.array(_X, dtype=float) / 255.0